In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from numpy.linalg import matrix_rank

# Load the data
df = pd.read_excel("/content/Preventative Health Care.xlsx", sheet_name="Data1")

# Drop rows with missing target
df = df.dropna(subset=["Health Ranking"])

# Define selected features
selected_columns = [
    "Preventable Hospitalization Rate", "number_of_fee_for_service_beneficiaries",
    "number_of_providers", "average_number_of_users_per_provider", "percentage_of_users_out_of_ffs_beneficiaries",
    "number_of_users", "average_number_of_providers_per_county", "number_of_dual_eligible_users",
    "percentage_of_dual_eligible_users_out_of_total_users", "Deaths", "% Fair or Poor Health",
    "Average Number of Physically Unhealthy Days", "Average Number of Mentally Unhealthy Days",
    "% Adults Reporting Currently Smoking", "% Adults with Obesity", "% Excessive Drinking", "% Uninsured",
    "# Primary Care Physicians", "Primary Care Physicians Rate", "# Mental Health Providers",
    "Mental Health Provider Rate", "total_payment"
]

# Filter and drop missing
df_model = df[selected_columns + ["Health Ranking"]].dropna()

# Define features and target
X = df_model.drop(columns=["Health Ranking"])
y = df_model["Health Ranking"]

# Logistic Regression using statsmodels (with collinearity fix)
X_const = sm.add_constant(X)

def remove_collinear_columns(X):
    while matrix_rank(X.values) < X.shape[1]:
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        max_corr_col = upper.sum().idxmax()
        print(f"Dropping collinear column: {max_corr_col}")
        X = X.drop(columns=[max_corr_col])
    return X

X_const = remove_collinear_columns(X_const)

logit_model = sm.Logit(y, X_const).fit()

# Display logistic regression summary
print("\n=== Logistic Regression Summary ===")
print(logit_model.summary())

# Train-test split for scikit-learn models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on the same X (or use train/test split if desired)
y_pred_probs = logit_model.predict(X_const)
y_pred_class = (y_pred_probs >= 0.5).astype(int)

# Logistic regression performance metrics
print("\n=== Logistic Regression Performance ===")
print(f"Accuracy: {accuracy_score(y, y_pred_class):.2f}")
print(f"Precision: {precision_score(y, y_pred_class):.2f}")
print(f"Recall: {recall_score(y, y_pred_class):.2f}")
print(f"F1 Score: {f1_score(y, y_pred_class):.2f}")

# Train and evaluate Random Forest
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print("\n=== Random Forest Results ===")
print(f"Accuracy: {accuracy_score(y_test, rf_pred):.2f}")
print("Classification Report:\n", classification_report(y_test, rf_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_pred))

# Train and evaluate Gradient Boosting
gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
print("\n=== Gradient Boosting Results ===")
print(f"Accuracy: {accuracy_score(y_test, gb_pred):.2f}")
print("Classification Report:\n", classification_report(y_test, gb_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, gb_pred))


Dropping collinear column: total_payment
Dropping collinear column: # Mental Health Providers
Dropping collinear column: # Primary Care Physicians
Dropping collinear column: % Adults Reporting Currently Smoking
Dropping collinear column: Deaths
Dropping collinear column: % Adults with Obesity
Dropping collinear column: average_number_of_providers_per_county
Optimization terminated successfully.
         Current function value: 0.491488
         Iterations 7

=== Logistic Regression Summary ===
                           Logit Regression Results                           
Dep. Variable:         Health Ranking   No. Observations:                 2633
Model:                          Logit   Df Residuals:                     2617
Method:                           MLE   Df Model:                           15
Date:                Sun, 13 Apr 2025   Pseudo R-squ.:                  0.2907
Time:                        15:37:32   Log-Likelihood:                -1294.1
converged:                 

In [ ]:
# Fit logistic regression model
logit_model = sm.Logit(y, X_const).fit()

# Print full summary
print("\n=== Full Logistic Regression Summary ===")
print(logit_model.summary())

# Extract p-values
p_values = logit_model.pvalues

# Filter to keep only variables with 0.01 <= p <= 0.05
valid_columns = p_values[ (p_values <= 0.05)].index.tolist()

# Refit the model with only those columns
X_filtered = X_const[valid_columns]
filtered_model = sm.Logit(y, X_filtered).fit()

# Print new summary
print("\n=== Filtered Logistic Regression Summary ( p ≤ 0.05) ===")
print(filtered_model.summary())


Optimization terminated successfully.
         Current function value: 0.491488
         Iterations 7

=== Full Logistic Regression Summary ===
                           Logit Regression Results                           
Dep. Variable:         Health Ranking   No. Observations:                 2633
Model:                          Logit   Df Residuals:                     2617
Method:                           MLE   Df Model:                           15
Date:                Sun, 13 Apr 2025   Pseudo R-squ.:                  0.2907
Time:                        15:39:36   Log-Likelihood:                -1294.1
converged:                       True   LL-Null:                       -1824.6
Covariance Type:            nonrobust   LLR p-value:                1.152e-216
                                                           coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score
)
import statsmodels.api as sm

# Load your dataset
df = pd.read_excel("/content/Preventative Health Care.xlsx", sheet_name="Data1")

# Drop rows with missing target
df = df.dropna(subset=["Health Ranking"])

# Select relevant features
columns_to_keep = [
    "Health Ranking",
    "Preventable Hospitalization Rate",
    "number_of_fee_for_service_beneficiaries",
    "number_of_providers",
    "average_number_of_users_per_provider",
    "percentage_of_users_out_of_ffs_beneficiaries",
    "number_of_users",
    "number_of_dual_eligible_users",
    "percentage_of_dual_eligible_users_out_of_total_users",
    "Deaths",
    "% Fair or Poor Health",
    "Average Number of Physically Unhealthy Days",
    "Average Number of Mentally Unhealthy Days",
    "% Adults Reporting Currently Smoking",
    "% Adults with Obesity",
    "% Excessive Drinking",
    "% Uninsured",
    "# Primary Care Physicians",
    "Primary Care Physicians Rate",
    "# Mental Health Providers",
    "Mental Health Provider Rate",
    "total_payment"
]

df = df[columns_to_keep]

# Drop rows with missing values
df = df.dropna()

# Drop previously identified non-significant variables
columns_to_drop = [
    "Preventable Hospitalization Rate",
    "number_of_providers",
    "number_of_dual_eligible_users",
    "Mental Health Provider Rate"
]
df = df.drop(columns=columns_to_drop)

# Log transform skewed variables
df["log_number_of_users"] = np.log1p(df["number_of_users"])
df["log_fee_service_beneficiaries"] = np.log1p(df["number_of_fee_for_service_beneficiaries"])

# Drop original versions of log-transformed columns
df = df.drop(columns=["number_of_users", "number_of_fee_for_service_beneficiaries"])

# Add interaction and polynomial terms
df["interaction_poorhealth_unhealthy"] = df["% Fair or Poor Health"] * df["Average Number of Physically Unhealthy Days"]
df["physically_unhealthy_squared"] = df["Average Number of Physically Unhealthy Days"] ** 2

# Define features and target
X = df.drop(columns=["Health Ranking"])
y = df["Health Ranking"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ======== Model 1: Regular Logistic Regression with statsmodels =========
X_train_sm = sm.add_constant(pd.DataFrame(X_train_scaled, columns=X.columns))
# Ensure indices are aligned by resetting them
X_train_sm = sm.add_constant(pd.DataFrame(X_train_scaled, columns=X.columns)).reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Fit the logistic regression model
logit_model = sm.Logit(y_train, X_train_sm).fit()
print("\n=== Logistic Regression Summary (statsmodels) ===")
print(logit_model.summary())



# ======== Model 2: Logistic Regression with sklearn =========
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

print("\n=== Logistic Regression (sklearn) Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# ======== Model 3: L1-Regularized Logistic Regression =========
clf_l1 = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
clf_l1.fit(X_train_scaled, y_train)
y_pred_l1 = clf_l1.predict(X_test_scaled)

print("\n=== L1 Logistic Regression Results ===")
print("Accuracy:", accuracy_score(y_test, y_pred_l1))
print("Precision:", precision_score(y_test, y_pred_l1))
print("Recall:", recall_score(y_test, y_pred_l1))
print("F1 Score:", f1_score(y_test, y_pred_l1))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_l1))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_l1))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_l1))


Optimization terminated successfully.
         Current function value: 0.439816
         Iterations 8

=== Logistic Regression Summary (statsmodels) ===
                           Logit Regression Results                           
Dep. Variable:         Health Ranking   No. Observations:                 2106
Model:                          Logit   Df Residuals:                     2086
Method:                           MLE   Df Model:                           19
Date:                Sun, 13 Apr 2025   Pseudo R-squ.:                  0.3653
Time:                        22:27:00   Log-Likelihood:                -926.25
converged:                       True   LL-Null:                       -1459.4
Covariance Type:            nonrobust   LLR p-value:                3.722e-214
                                                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import pandas as pd
import numpy as np

# Train/test split assumed already done:
# X_train, X_test, y_train, y_test

# Fit L1-regularized logistic regression
model = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Output results
print("=== L1 Logistic Regression Results ===")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC Score: {roc_auc}")
print("\nClassification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

# Coefficient summary
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_[0]
})
coef_df['Abs_Coefficient'] = np.abs(coef_df['Coefficient'])
coef_df = coef_df.sort_values(by='Abs_Coefficient', ascending=False)
print("\nTop Non-zero Coefficients:\n", coef_df[coef_df['Coefficient'] != 0])


=== L1 Logistic Regression Results ===
Accuracy: 0.7988614800759013
Precision: 0.7923076923076923
Recall: 0.7984496124031008
F1 Score: 0.7953667953667953
ROC AUC Score: 0.8727702371689577

Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.80      0.80       269
           1       0.79      0.80      0.80       258

    accuracy                           0.80       527
   macro avg       0.80      0.80      0.80       527
weighted avg       0.80      0.80      0.80       527

Confusion Matrix:
 [[215  54]
 [ 52 206]]

Top Non-zero Coefficients:
                                               Feature   Coefficient  \
2   percentage_of_dual_eligible_users_out_of_total...  1.065057e+01   
1        percentage_of_users_out_of_ffs_beneficiaries -3.383891e+00   
6           Average Number of Mentally Unhealthy Days -1.211712e+00   
7                % Adults Reporting Currently Smoking  2.965011e-01   
15                                

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create an L1-regularized logistic regression model
logreg_l1 = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)

# Fit the model on the training data
logreg_l1.fit(X_train_scaled, y_train)

# Predict on the test data
y_pred_l1 = logreg_l1.predict(X_test_scaled)

# Calculate metrics
accuracy = logreg_l1.score(X_test_scaled, y_test)
precision = precision_score(y_test, y_pred_l1)
recall = recall_score(y_test, y_pred_l1)
f1 = f1_score(y_test, y_pred_l1)
roc_auc = roc_auc_score(y_test, logreg_l1.predict_proba(X_test_scaled)[:, 1])

# Display the results
print("=== L1 Logistic Regression Results ===")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC AUC Score: {roc_auc}")

# You can also display the confusion matrix and classification report if needed
from sklearn.metrics import classification_report, confusion_matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred_l1))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_l1))


=== L1 Logistic Regression Results ===
Accuracy: 0.8140417457305503
Precision: 0.810077519379845
Recall: 0.810077519379845
F1 Score: 0.810077519379845
ROC AUC Score: 0.8780582692141436

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.82      0.82       269
           1       0.81      0.81      0.81       258

    accuracy                           0.81       527
   macro avg       0.81      0.81      0.81       527
weighted avg       0.81      0.81      0.81       527


Confusion Matrix:
[[220  49]
 [ 49 209]]


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# === 1. Load Data ===
df = pd.read_excel("/content/Preventative Health Care.xlsx", sheet_name="Data1")

# Convert 'Health Ranking' into binary if it's not already (0 = good, 1 = poor)
df = df[df['Health Ranking'].isin([0, 1])]  # keep only binary classes

# === 2. Select features and target ===
features = [
    'Preventable Hospitalization Rate', 'number_of_fee_for_service_beneficiaries',
    'number_of_providers', 'average_number_of_users_per_provider',
    'percentage_of_users_out_of_ffs_beneficiaries', 'number_of_users',
    'number_of_dual_eligible_users', 'percentage_of_dual_eligible_users_out_of_total_users',
    '% Fair or Poor Health', 'Average Number of Physically Unhealthy Days',
    'Average Number of Mentally Unhealthy Days', '% Adults Reporting Currently Smoking',
    '% Adults with Obesity', '% Excessive Drinking', '% Uninsured',
    '# Primary Care Physicians', 'Primary Care Physicians Rate',
    '# Mental Health Providers', 'Mental Health Provider Rate', 'total_payment'
]

X = df[features].copy()
y = df['Health Ranking']

# Drop columns with too many missing values
X = X.dropna(axis=1, thresh=0.8 * len(X))

# Impute missing with median
X = X.fillna(X.median())

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# === 3. Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, test_size=0.2, stratify=y, random_state=42
)

# === 4. Model Performance using scikit-learn ===
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("\n=== Model Evaluation ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# === 5. Detailed Coefficient Analysis with statsmodels ===
X_train_sm = sm.add_constant(X_train)
logit_model = sm.Logit(y_train, X_train_sm).fit()
print("\n=== Logistic Regression Summary ===")
print(logit_model.summary())

# === 6. Interpretation of Key Drivers ===
print("\n=== Key Drivers (p ≤ 0.05) ===")
coefs = logit_model.params
pvals = logit_model.pvalues

key_drivers = pd.DataFrame({
    "Feature": coefs.index,
    "Coefficient": coefs.values,
    "P-Value": pvals.values
})

key_drivers = key_drivers[key_drivers['P-Value'] <= 0.05].sort_values(by='P-Value')
print(key_drivers)

# === 7. Refit model using only significant features ===
significant_features = key_drivers[key_drivers["Feature"] != "const"]["Feature"].tolist()
X_train_reduced = X_train[significant_features]
X_train_reduced_sm = sm.add_constant(X_train_reduced)

logit_model_reduced = sm.Logit(y_train, X_train_reduced_sm).fit()
print("\n=== Reduced Logistic Regression Summary ===")
print(logit_model_reduced.summary())

X_test_reduced = X_test[significant_features]
X_test_reduced_sm = sm.add_constant(X_test_reduced)
y_pred_reduced = logit_model_reduced.predict(X_test_reduced_sm)
y_pred_class = (y_pred_reduced >= 0.5).astype(int)

print("\n=== Reduced Model Evaluation ===")
print("Accuracy:", accuracy_score(y_test, y_pred_class))
print("Classification Report:\n", classification_report(y_test, y_pred_class))


# Optional: Interpretations
print("\n=== Interpretation ===")
for _, row in key_drivers.iterrows():
    direction = "increase" if row['Coefficient'] > 0 else "decrease"
    print(f"An increase in '{row['Feature']}' is associated with a(n) {direction} in the odds of being in the worse health ranking.")



=== Model Evaluation ===
Accuracy: 0.7882165605095541
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.81      0.80       322
           1       0.79      0.77      0.78       306

    accuracy                           0.79       628
   macro avg       0.79      0.79      0.79       628
weighted avg       0.79      0.79      0.79       628

Confusion Matrix:
 [[260  62]
 [ 71 235]]
Optimization terminated successfully.
         Current function value: 0.460421
         Iterations 8

=== Logistic Regression Summary ===
                           Logit Regression Results                           
Dep. Variable:         Health Ranking   No. Observations:                 2510
Model:                          Logit   Df Residuals:                     2489
Method:                           MLE   Df Model:                           20
Date:                Sun, 13 Apr 2025   Pseudo R-squ.:                  0.3355
Time:             

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# === 1. Load Data ===
df = pd.read_excel("/content/Preventative Health Care.xlsx", sheet_name="Data1")

# Convert 'Health Ranking' into binary if it's not already (0 = good, 1 = poor)
df = df[df['Health Ranking'].isin([0, 1])]  # keep only binary classes

# === 2. Select features and target ===
features = [
    'Preventable Hospitalization Rate', 'number_of_fee_for_service_beneficiaries',
    'number_of_providers', 'average_number_of_users_per_provider',
    'percentage_of_users_out_of_ffs_beneficiaries', 'number_of_users',
    'number_of_dual_eligible_users', 'percentage_of_dual_eligible_users_out_of_total_users',
    '% Fair or Poor Health', 'Average Number of Physically Unhealthy Days',
    'Average Number of Mentally Unhealthy Days', '% Adults Reporting Currently Smoking',
    '% Adults with Obesity', '% Excessive Drinking', '% Uninsured',
    '# Primary Care Physicians', 'Primary Care Physicians Rate',
    '# Mental Health Providers', 'Mental Health Provider Rate', 'total_payment'
]

X = df[features].copy()
y = df['Health Ranking']

# Drop columns with too many missing values
X = X.dropna(axis=1, thresh=0.8 * len(X))

# Impute missing with median
X = X.fillna(X.median())

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# === 3. Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y, test_size=0.2, stratify=y, random_state=42
)

# === 4. Logistic Regression (Full Model) ===
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)

# === 5. Detailed Coefficient Analysis with statsmodels ===
X_train_sm = sm.add_constant(X_train)
logit_model = sm.Logit(y_train, X_train_sm).fit()
print("\n=== Logistic Regression Summary ===")
print(logit_model.summary())

# === 6. Interpretation of Key Drivers ===
coefs = logit_model.params
pvals = logit_model.pvalues
key_drivers = pd.DataFrame({
    "Feature": coefs.index,
    "Coefficient": coefs.values,
    "P-Value": pvals.values
})
key_drivers = key_drivers[key_drivers['P-Value'] <= 0.05].sort_values(by='P-Value')
print("\n=== Key Drivers (p ≤ 0.05) ===")
print(key_drivers)

# === 7. Refit model using only significant features ===
significant_features = key_drivers[key_drivers["Feature"] != "const"]["Feature"].tolist()
X_train_reduced = X_train[significant_features]
X_train_reduced_sm = sm.add_constant(X_train_reduced)

logit_model_reduced = sm.Logit(y_train, X_train_reduced_sm).fit()
print("\n=== Reduced Logistic Regression Summary ===")
print(logit_model_reduced.summary())

X_test_reduced = X_test[significant_features]
X_test_reduced_sm = sm.add_constant(X_test_reduced)
y_pred_reduced = logit_model_reduced.predict(X_test_reduced_sm)
y_pred_class = (y_pred_reduced >= 0.5).astype(int)

print("\n=== Reduced Logistic Regression Evaluation ===")
print("Accuracy:", accuracy_score(y_test, y_pred_class))
print("Classification Report:\n", classification_report(y_test, y_pred_class))

# === 8. Refit Random Forest and Gradient Boosting on Reduced Features ===

# Random Forest (Reduced Model)
rf_reduced = RandomForestClassifier(random_state=42)
rf_reduced.fit(X_train_reduced, y_train)
y_pred_rf_reduced = rf_reduced.predict(X_test_reduced)

# Gradient Boosting (Reduced Model)
gb_reduced = GradientBoostingClassifier(random_state=42)
gb_reduced.fit(X_train_reduced, y_train)
y_pred_gb_reduced = gb_reduced.predict(X_test_reduced)

# Print results for reduced models
print("\n=== Reduced Random Forest Evaluation ===")
print("Accuracy:", accuracy_score(y_test, y_pred_rf_reduced))
print("Classification Report:\n", classification_report(y_test, y_pred_rf_reduced))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_rf_reduced))

print("\n=== Reduced Gradient Boosting Evaluation ===")
print("Accuracy:", accuracy_score(y_test, y_pred_gb_reduced))
print("Classification Report:\n", classification_report(y_test, y_pred_gb_reduced))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_gb_reduced))


Optimization terminated successfully.
         Current function value: 0.460421
         Iterations 8

=== Logistic Regression Summary ===
                           Logit Regression Results                           
Dep. Variable:         Health Ranking   No. Observations:                 2510
Model:                          Logit   Df Residuals:                     2489
Method:                           MLE   Df Model:                           20
Date:                Thu, 17 Apr 2025   Pseudo R-squ.:                  0.3355
Time:                        14:26:57   Log-Likelihood:                -1155.7
converged:                       True   LL-Null:                       -1739.1
Covariance Type:            nonrobust   LLR p-value:                9.175e-235
                                                           coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------

In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import statsmodels.api as sm

# STEP 1: Load data
# STEP 2: Keep only numeric features and extract binary target 'Health Ranking'
numeric_df = df.select_dtypes(include=[np.number])
X = numeric_df.drop(columns=['Health Ranking'])
for col in [ 'FIPS_Code']:
    if col in X.columns:
        X = X.drop(columns=[col])
y = df['Health Ranking']

# STEP 3: Impute missing values with -1 AND add missingness indicators
missing_ind = X.isna().astype(int).add_suffix('_missing')
imputer = SimpleImputer(strategy='constant', fill_value=-1)
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns, index=X.index)
X_aug = pd.concat([X_imputed, missing_ind], axis=1)

# STEP 4: Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_aug)
X_scaled_df = pd.DataFrame(X_scaled, columns=X_aug.columns)

# Drop highly correlated features
def drop_high_corr_features(df, threshold=0.99):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return df.drop(columns=to_drop)

X_scaled_df_dedup = drop_high_corr_features(X_scaled_df)
X_scaled_df_dedup = X_scaled_df_dedup.loc[:, X_scaled_df_dedup.std() > 1e-6]  # drop near-constant columns

# STEP 4.5: Backward Elimination Function
def backward_elimination(X, y, sl=0.05):
    X_ = sm.add_constant(X.copy())
    while True:
        model = sm.Logit(y, X_).fit(disp=False)
        p_values = model.pvalues
        max_p = p_values.max()
        if max_p > sl:
            worst_feature = p_values.idxmax()
            print(f"Removing '{worst_feature}' with p = {max_p:.4f}")
            X_ = X_.drop(columns=[worst_feature])
        else:
            break
    final_model = sm.Logit(y, X_).fit()
    return final_model, X_

# STEP 4.6: Apply Backward Elimination
logit_model_optimized, X_selected = backward_elimination(X_scaled_df_dedup, y)

print("\n=== Logistic Regression Summary (Backward Elimination) ===")
print(logit_model_optimized.summary())

# STEP 5: PCA (retain 95% of variance)
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# STEP 6: Train/Test split (stratify to preserve class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

# STEP 8: Define models
models = {
    "Logistic Regression": LogisticRegression(
        solver='liblinear',
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    ),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# STEP 9: Train, predict, and evaluate
def evaluate(name, model, X_tr, y_tr, X_te, y_te):
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(y_te, y_pred))
    print("Classification Report:\n", classification_report(y_te, y_pred, digits=3))
    print("Confusion Matrix:\n", confusion_matrix(y_te, y_pred))

for name, mdl in models.items():
    evaluate(name, mdl, X_train, y_train, X_test, y_test)


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Removing '% Fair or Poor Health_missing' with p = 1.0000
Removing 'number_of_dual_eligible_users' with p = 0.6728
Removing 'average_number_of_users_per_provider_missing' with p = 0.5804
Removing '# Primary Care Physicians_missing' with p = 0.4754
Removing 'number_of_providers' with p = 0.4143
Removing 'Deaths_missing' with p = 0.1935
Removing '# Mental Health Providers_missing' with p = 0.3309
Removing 'Primary Care Physicians Rate' with p = 0.0945
Removing 'Mental Health Provider Rate' with p = 0.2019
Removing 'number_of_fee_for_service_beneficiaries' with p = 0.0689
Removing '% Fair or Poor Health' with p = 0.0586
Optimization terminated successfully.
         Current function value: 0.455218
         Iterations 8

=== Logistic Regression Summary (Backward Elimination) ===
                           Logit Regression Results                           
Dep. Variable:         Health Ranking   No. Observations:                 3138
Model:                          Logit   Df Residuals:   

In [5]:
# Assume X_scaled_df_dedup is ready from previous steps
X = X_scaled_df_dedup
y = df['Health Ranking']

# Add constant column for intercept
X = sm.add_constant(X)

In [6]:
def forward_selection(X, y, threshold_in=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    selected_features = ['const']

    while remaining_features:
        scores_with_candidates = []
        for candidate in remaining_features:
            if candidate == 'const' or candidate in selected_features:
                continue
            try:
                model = sm.Logit(y, X[selected_features + [candidate]]).fit(disp=0)
                pval = model.pvalues[candidate]
                scores_with_candidates.append((pval, candidate))
            except Exception as e:
                continue

        if not scores_with_candidates:
            break

        scores_with_candidates.sort()
        best_pval, best_candidate = scores_with_candidates[0]
        if best_pval < threshold_in:
            selected_features.append(best_candidate)
            remaining_features.remove(best_candidate)
        else:
            break

    return selected_features

In [7]:
def stepwise_selection(X, y, threshold_in=0.05, threshold_out=0.05):
    included = ['const']
    while True:
        changed = False

        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pvals = pd.Series(index=excluded)
        for new_col in excluded:
            try:
                model = sm.Logit(y, X[included + [new_col]]).fit(disp=0)
                new_pvals[new_col] = model.pvalues[new_col]
            except Exception:
                continue
        if not new_pvals.empty:
            best_pval = new_pvals.min()
            if best_pval < threshold_in:
                best_feature = new_pvals.idxmin()
                included.append(best_feature)
                changed = True

        # Backward step
        model = sm.Logit(y, X[included]).fit(disp=0)
        pvals = model.pvalues.iloc[1:]  # skip intercept
        worst_pval = pvals.max()
        if worst_pval > threshold_out:
            worst_feature = pvals.idxmax()
            included.remove(worst_feature)
            changed = True

        if not changed:
            break
    return included


In [8]:
 # Forward Selection
selected_forward = forward_selection(X, y)
print("Selected Features (Forward):", selected_forward)

# Stepwise Selection
selected_stepwise = stepwise_selection(X, y)
print("Selected Features (Stepwise):", selected_stepwise)

# Refit models
logit_forward = sm.Logit(y, X[selected_forward]).fit()
print("\n=== Logistic Regression Summary (Forward Selection) ===")
print(logit_forward.summary())

logit_stepwise = sm.Logit(y, X[selected_stepwise]).fit()
print("\n=== Logistic Regression Summary (Stepwise Selection) ===")
print(logit_stepwise.summary())

/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood op

Selected Features (Forward): ['const', '% Adults Reporting Currently Smoking', 'percentage_of_users_out_of_ffs_beneficiaries', '% Adults with Obesity', 'average_number_of_users_per_provider', 'percentage_of_dual_eligible_users_out_of_total_users', 'number_of_dual_eligible_users_missing', 'Average Number of Mentally Unhealthy Days', '% Excessive Drinking', 'Average Number of Physically Unhealthy Days', 'Preventable Hospitalization Rate_missing', '% Uninsured', 'Primary Care Physicians Rate', 'Mental Health Provider Rate', 'Preventable Hospitalization Rate']


/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood op

Selected Features (Stepwise): ['const', '% Adults Reporting Currently Smoking', 'percentage_of_users_out_of_ffs_beneficiaries', '% Adults with Obesity', 'average_number_of_users_per_provider', 'percentage_of_dual_eligible_users_out_of_total_users', 'number_of_dual_eligible_users_missing', 'Average Number of Mentally Unhealthy Days', '% Excessive Drinking', 'Average Number of Physically Unhealthy Days', 'Preventable Hospitalization Rate_missing', '% Uninsured', 'Primary Care Physicians Rate', 'Mental Health Provider Rate', 'Preventable Hospitalization Rate']
Optimization terminated successfully.
         Current function value: 0.459722
         Iterations 7

=== Logistic Regression Summary (Forward Selection) ===
                           Logit Regression Results                           
Dep. Variable:         Health Ranking   No. Observations:                 3138
Model:                          Logit   Df Residuals:                     3123
Method:                           MLE   

In [12]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import statsmodels.api as sm

# STEP 1: Load data
# df = pd.read_excel("...")  # Assuming already loaded

# STEP 2: Keep only numeric features and extract binary target 'Health Ranking'
numeric_df = df.select_dtypes(include=[np.number])
X = numeric_df.drop(columns=['Health Ranking'])
for col in ['FIPS_Code']:
    if col in X.columns:
        X = X.drop(columns=[col])
y = df['Health Ranking']

# STEP 3: Impute missing values with -1 AND add missingness indicators
missing_ind = X.isna().astype(int).add_suffix('_missing')
imputer = SimpleImputer(strategy='constant', fill_value=-1)
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns, index=X.index)
X_aug = pd.concat([X_imputed, missing_ind], axis=1)


# STEP 3.5: Remove specific columns and all *_missing columns
columns_to_remove = [
    '# Mental Health Providers',
    'Average Number of Mentally Unhealthy Days',
    'Preventable Hospitalization Rate_missing',
    'number_of_dual_eligible_users_missing',
    'Mental Health Provider Rate'  # cleaned this name
]

# Remove listed columns if they exist
X_aug = X_aug.drop(columns=[col for col in columns_to_remove if col in X_aug.columns])

# Remove all *_missing columns
X_aug = X_aug.drop(columns=[col for col in X_aug.columns if col.endswith('_missing')])



# STEP 4: Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_aug)
X_scaled_df = pd.DataFrame(X_scaled, columns=X_aug.columns)

# Drop highly correlated features
def drop_high_corr_features(df, threshold=0.99):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return df.drop(columns=to_drop)

X_scaled_df_dedup = drop_high_corr_features(X_scaled_df)
X_scaled_df_dedup = X_scaled_df_dedup.loc[:, X_scaled_df_dedup.std() > 1e-6]

# STEP 4.5: Backward Elimination
def backward_elimination(X, y, sl=0.05):
    X_ = sm.add_constant(X.copy())
    while True:
        model = sm.Logit(y, X_).fit(disp=False)
        p_values = model.pvalues
        max_p = p_values.max()
        if max_p > sl:
            worst_feature = p_values.idxmax()
            print(f"Removing '{worst_feature}' with p = {max_p:.4f}")
            X_ = X_.drop(columns=[worst_feature])
        else:
            break
    final_model = sm.Logit(y, X_).fit()
    return final_model, X_

# STEP 4.6: Apply Backward Elimination
logit_model_optimized, X_selected = backward_elimination(X_scaled_df_dedup, y)

print("\n=== Logistic Regression Summary (Backward Elimination) ===")
print(logit_model_optimized.summary())

# STEP 5: PCA (retain 95% of variance)
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_scaled_df_dedup)

# STEP 6: Train/Test split (stratify to preserve class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

# STEP 8: Define models
models = {
    "Logistic Regression": LogisticRegression(
        solver='liblinear',
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    ),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# STEP 9: Train, predict, and evaluate
def evaluate(name, model, X_tr, y_tr, X_te, y_te):
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_te)
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(y_te, y_pred))
    print("Classification Report:\n", classification_report(y_te, y_pred, digits=3))
    print("Confusion Matrix:\n", confusion_matrix(y_te, y_pred))

for name, mdl in models.items():
    evaluate(name, mdl, X_train, y_train, X_test, y_test)


Removing 'number_of_fee_for_service_beneficiaries' with p = 0.7557
Removing 'number_of_providers' with p = 0.3002
Removing 'Primary Care Physicians Rate' with p = 0.0807
Optimization terminated successfully.
         Current function value: 0.482797
         Iterations 8

=== Logistic Regression Summary (Backward Elimination) ===
                           Logit Regression Results                           
Dep. Variable:         Health Ranking   No. Observations:                 3138
Model:                          Logit   Df Residuals:                     3124
Method:                           MLE   Df Model:                           13
Date:                Thu, 17 Apr 2025   Pseudo R-squ.:                  0.3032
Time:                        17:05:35   Log-Likelihood:                -1515.0
converged:                       True   LL-Null:                       -2174.2
Covariance Type:            nonrobust   LLR p-value:                6.031e-274
                                    